In [ ]:
import os
import tensorflow as tf
if tf.__version__!='2.11.0':
  !pip install tensorflow==2.11.0 --quiet
  os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 62.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.


In [ ]:
!pip install keras-rl2 --quiet
!pip install gym[atari] --quiet
!pip install atari-py --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m atari_py.import_roms /content/drive/MyDrive/dqn/roms/

copying breakout.bin from /content/drive/MyDrive/dqn/roms/Breakout - Breakaway IV.bin to /usr/local/lib/python3.10/dist-packages/atari_py/atari_roms/breakout.bin


In [ ]:
import gym
from gym.envs.registration import register

/usr/local/lib/python3.10/dist-packages/ale_py/roms/__init__.py:89: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  ROMS = resolve_roms()


In [ ]:
register(
    id='Breakout-v4',
    entry_point='gym.envs.atari:AtariEnv',
    kwargs={'game': 'breakout', 'obs_type': 'image', 'frameskip': 1},
    max_episode_steps=10000,
    nondeterministic=False,
)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:542: UserWarning: WARN: Overriding environment Breakout-v4
  logger.warn(f"Overriding environment {spec.id}")


In [ ]:
from __future__ import division
import argparse

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers.legacy import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint


INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4


class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        # We could perform this processing step in `process_observation`. In this case, however,
        # we would need to store a `float32` array instead, which is 4x more memory intensive than
        # an `uint8` array. This matters if we store 1M observations.
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

# parser = argparse.ArgumentParser()
# parser.add_argument('--mode', choices=['train', 'test'], default='train')
# parser.add_argument('--env-name', type=str, default='BreakoutDeterministic-v4')
# parser.add_argument('--weights', type=str, default=None)
# args = parser.parse_args()

env_name = 'Breakout-v4'
mode = 'train'
weights = None

# Get the environment and extract the number of actions.
env = gym.make(env_name)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Next, we build our model. We use the same model that was described by Mnih et al. (2015).
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()

# (width, height, channels)
model.add(Permute((2, 3, 1), input_shape=input_shape))

model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in tensorflow.keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()

# Select a policy. We use eps-greedy action selection, which means that a random action is selected
# with probability eps. We anneal eps from 1.0 to 0.1 over the course of 1M steps. This is done so that
# the agent initially explores the environment (high eps) and then gradually sticks to what it knows
# (low eps). We also set a dedicated eps value that is used during testing. Note that we set it to 0.05
# so that the agent still performs some random actions. This ensures that the agent cannot get stuck.
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000000)

# The trade-off between exploration and exploitation is difficult and an on-going research topic.
# If you want, you can experiment with the parameters or use a different policy. Another popular one
# is Boltzmann-style exploration:
# policy = BoltzmannQPolicy(tau=1.)
# Feel free to give it a try!

dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
               train_interval=4, delta_clip=1.)
dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])

/usr/local/lib/python3.10/dist-packages/gym/envs/atari/environment.py:68: UserWarning: WARN: obs_type "image" should be replaced with the image type, one of: rgb, grayscale
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute (Permute)           (None, 84, 84, 4)         0         
                                                                 
 conv2d (Conv2D)             (None, 20, 20, 32)        8224      
                                                                 
 activation (Activation)     (None, 20, 20, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 activation_1 (Activation)   (None, 9, 9, 64)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 activation_2 (Activation)   (None, 7, 7, 64)          0

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
# Okay, now it's time to learn something! We capture the interrupt exception so that training
# can be prematurely aborted. Notice that now you can use the built-in tensorflow.keras callbacks!
weights_filename = f'/content/drive/MyDrive/dqn/dqn_{env_name}_weights.h5f'
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = f'/content/drive/MyDrive/dqn/dqn_{env_name}_log.json'
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]
dqn.fit(env, callbacks=callbacks, nb_steps=1750000, log_interval=10000, verbose = 2)

# After training is done, we save the final weights one more time.
dqn.save_weights(weights_filename, overwrite=True)

# Finally, evaluate our algorithm for 10 episodes.
dqn.test(env, nb_episodes=10, visualize=False)

Training for 1750000 steps ...


/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


     809/1750000: episode: 1, duration: 11.261s, episode steps: 809, steps per second:  72, episode reward:  2.000, mean reward:  0.002 [ 0.000,  1.000], mean action: 1.533 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    1305/1750000: episode: 2, duration: 1.821s, episode steps: 496, steps per second: 272, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.510 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    2765/1750000: episode: 3, duration: 8.969s, episode steps: 1460, steps per second: 163, episode reward:  5.000, mean reward:  0.003 [ 0.000,  1.000], mean action: 1.546 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    3263/1750000: episode: 4, duration: 3.553s, episode steps: 498, steps per second: 140, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.556 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    3974/1750000: episode: 5, duration: 2.695s, episode steps: 711

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   50450/1750000: episode: 69, duration: 16.527s, episode steps: 496, steps per second:  30, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.464 [0.000, 3.000],  loss: 0.001006, mae: 0.036553, mean_q: -0.030346, mean_eps: 0.954797
   51294/1750000: episode: 70, duration: 27.740s, episode steps: 844, steps per second:  30, episode reward:  2.000, mean reward:  0.002 [ 0.000,  1.000], mean action: 1.562 [0.000, 3.000],  loss: 0.000770, mae: 0.034965, mean_q: -0.037727, mean_eps: 0.954215
   51998/1750000: episode: 71, duration: 21.173s, episode steps: 704, steps per second:  33, episode reward:  1.000, mean reward:  0.001 [ 0.000,  1.000], mean action: 1.526 [0.000, 3.000],  loss: 0.001173, mae: 0.034489, mean_q: -0.037482, mean_eps: 0.953519
   52810/1750000: episode: 72, duration: 26.656s, episode steps: 812, steps per second:  30, episode reward:  2.000, mean reward:  0.002 [ 0.000,  1.000], mean action: 1.458 [0.000, 3.000],  loss: 0.001551, mae: 0.034005

In [ ]:
weights_filename = f'/content/drive/MyDrive/dqn/dqn_{env_name}_weights.h5f'
if weights:
    weights_filename = weights
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=True)